## From Scikit-learn to Arduino

Machine learning on Arduino boards is it possible?

Do you want to run the model you trained in Python into any C++ project, be it Arduino, STM32, ESP32?

## Step1 load data
To train a classifier, we need some data. If you're starting from zero and don't have already a preferred folder structure, I suggest you to create a folder that will hold the data you collect.

Inside this folder, create a dedicated file (.csv) for each of the classes you want to classify, putting a sample on each line. If doing so, you can use the next function to load this data.

In [1]:
#!mkdir datafolder

In [2]:
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/sample_binarydata.csv -O /content/datafolder/sample_binarydata.csv

In [3]:
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/sample_binarydata.csv

In [4]:
import numpy as np

def load_data(file):
    data = np.loadtxt(file, dtype=float, delimiter=',')
    classmap = {
                0:'survived',
                1:'died',
                }
    return data, classmap

In [5]:
path='sample_binarydata.csv'
data = np.loadtxt(path, dtype=float, delimiter=',')
data

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

## Step2 Train Classifier

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

def modeling(data):
    X, y = data[:, :-1], data[:, -1]
    k = 5
    k_best = SelectKBest(score_func=f_classif, k=k)
    k_best.fit(X, y)
    selected_feature_indices = k_best.get_support(indices=True)

    clf = RandomForestClassifier(20, max_depth=10, random_state=667)

    return clf.fit(X[:, selected_feature_indices], y)

## Step3 Export to C
Now you can convert the trained classifier to plain C code using the micromlgen package.



In [7]:
from micromlgen import port

dataset, classmap = load_data(path)
classifier = modeling(dataset)
classmap

{0: 'survived', 1: 'died'}

In [8]:
## testing classifier prediction
#classifier.predict([[6.,148.,33.6,0.627,50.]])[0] ## class1.0
classifier.predict([[1., 85.,26.6,0.351,31.]])[0] ##  class0.0

0.0

In [9]:
c_code = port(classifier, classmap=classmap)
#print(c_code)

In [10]:
## write model into model.h
file_path = "model.h"

with open(file_path, 'w') as file:
    file.write(c_code)

## Step4 Load model into Arduino

In [11]:
# // put the code you got in Step 3 into this file
# #include "model.h"

# // this class will be different if you used another type of classifier, just check the model.h file
# Eloquent::ML::Port::RandomForest classifier;

# void classify() {
#     float x_sample[] = { /* fill this vector with sample values */ };

#     Serial.print("Predicted class: ");
#     Serial.println(classifier.predictLabel(x_sample));
# }